In [ ]:
import json
import glob
import numpy as np
import cv2
import os
import face_recognition
from tqdm.autonotebook import tqdm
import mediapipe as mp

class VideoFaceProcessor:
    def __init__(self):
        self.mp_face_detection = mp.solutions.face_detection
        self.face_detection = self.mp_face_detection.FaceDetection(
            model_selection=1, min_detection_confidence=0.5
        )
        
    def frame_extract(self, path, skip_frames=2):
        """Extract frames from video with frame skipping"""
        print(f"Opening video: {path}")
        vidObj = cv2.VideoCapture(path)
        
        if not vidObj.isOpened():
            print(f"Error: Could not open video {path}")
            return
            
        total_frames = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Total frames in video: {total_frames}")
        
        count = 0
        frames_yielded = 0
        
        while count < total_frames:
            success, image = vidObj.read()
            if not success:
                print(f"Failed to read frame {count}")
                break
                
            if count % skip_frames == 0:
                frames_yielded += 1
                yield image
                
            count += 1
            
        print(f"Finished extracting {frames_yielded} frames from {path}")
        vidObj.release()

    def detect_face_mediapipe(self, frame):
        """Detect faces using MediaPipe"""
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(frame_rgb)
        
        if results.detections:
            detection = results.detections[0]
            bbox = detection.location_data.relative_bounding_box
            
            h, w = frame.shape[:2]
            x, y = int(bbox.xmin * w), int(bbox.ymin * h)
            width, height = int(bbox.width * w), int(bbox.height * h)
            
            # Add padding to the face region
            padding = int(min(width, height) * 0.1)
            y = max(0, y - padding)
            x = max(0, x - padding)
            width = min(w - x, width + 2*padding)
            height = min(h - y, height + 2*padding)
            
            return [y, x + width, y + height, x]
        return None

    def create_face_videos(self, input_dir, output_dir, margin=0.3, 
                          max_frames=100, batch_size=16, skip_frames=2):
        """Process videos to extract face regions"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Get list of video files
        video_files = glob.glob(os.path.join(input_dir, '*.mp4'))
        print(f"Found {len(video_files)} videos in {input_dir}")
        
        if not video_files:
            print(f"No .mp4 files found in {input_dir}")
            return
        
        for video_path in tqdm(video_files, desc="Processing videos"):
            video_name = os.path.basename(video_path)
            out_path = os.path.join(output_dir, video_name)
            
            if os.path.exists(out_path):
                print(f"Skipping {video_name} - already exists")
                continue
            
            print(f"\nProcessing video: {video_name}")
            frames = []
            processed_frames = 0
            
            try:
                # Get video properties for output
                temp_cap = cv2.VideoCapture(video_path)
                if not temp_cap.isOpened():
                    print(f"Error: Could not open {video_path}")
                    continue
                    
                fps = int(temp_cap.get(cv2.CAP_PROP_FPS))
                temp_cap.release()
                
                # Initialize video writer
                out = cv2.VideoWriter(
                    out_path,
                    cv2.VideoWriter_fourcc(*'MJPG'),
                    fps,
                    (112, 112)
                )
                
                frame_generator = self.frame_extract(video_path, skip_frames)
                frames_processed = 0
                faces_found = 0
                
                for frame in frame_generator:
                    if frames_processed >= max_frames:
                        break
                    
                    # Detect face
                    face_location = self.detect_face_mediapipe(frame)
                    
                    if face_location:
                        faces_found += 1
                        top, right, bottom, left = face_location
                        try:
                            face_frame = frame[top:bottom, left:right]
                            face_frame = cv2.resize(face_frame, (112, 112))
                            out.write(face_frame)
                        except Exception as e:
                            print(f"Error processing face in frame {frames_processed}: {str(e)}")
                    
                    frames_processed += 1
                    
                out.release()
                
                print(f"Completed {video_name}:")
                print(f"- Processed {frames_processed} frames")
                print(f"- Found {faces_found} faces")
                
                # Verify output
                if os.path.exists(out_path):
                    if os.path.getsize(out_path) > 0:
                        print(f"Successfully saved to {out_path}")
                    else:
                        print(f"Warning: Output file is empty: {out_path}")
                        os.remove(out_path)
                else:
                    print(f"Error: Failed to create output file: {out_path}")
                
            except Exception as e:
                print(f"Error processing {video_name}: {str(e)}")
                if os.path.exists(out_path):
                    os.remove(out_path)
                continue

def main():
    # Set up paths
    input_path = "Dataset/FF++"
    output_path = "Dataset/FF_Face_only_data_processed"
    
    print("Starting video processing...")
    print(f"Input path: {input_path}")
    print(f"Output path: {output_path}")
    
    # Verify input directory exists
    if not os.path.exists(input_path):
        print(f"Error: Input directory {input_path} does not exist")
        return
        
    # Create output directories
    os.makedirs(os.path.join(output_path, "real"), exist_ok=True)
    os.makedirs(os.path.join(output_path, "fake"), exist_ok=True)
    
    # Initialize processor
    processor = VideoFaceProcessor()
    
    # Process real videos
    print("\nProcessing real videos...")
    real_path = os.path.join(input_path, "real")
    if os.path.exists(real_path):
        processor.create_face_videos(
            input_dir=real_path,
            output_dir=os.path.join(output_path, "real"),
            margin=0.3,
            max_frames=100,
            batch_size=16,
            skip_frames=2
        )
    else:
        print(f"Warning: Real videos directory not found at {real_path}")
    
    # Process fake videos
    print("\nProcessing fake videos...")
    fake_path = os.path.join(input_path, "fake")
    if os.path.exists(fake_path):
        processor.create_face_videos(
            input_dir=fake_path,
            output_dir=os.path.join(output_path, "fake"),
            margin=0.3,
            max_frames=100,
            batch_size=16,
            skip_frames=2
        )
    else:
        print(f"Warning: Fake videos directory not found at {fake_path}")

if __name__ == "__main__":
    main()

I0000 00:00:1739738048.649600  204170 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1739738048.676407  207057 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: D3D12 (AMD Radeon(TM) Graphics)


Starting video processing...
Input path: Dataset/FF++
Output path: Dataset/FF_Face_only_data_processed

Processing real videos...
Found 16 videos in Dataset/FF++/real


Processing videos: 100%|██████████| 16/16 [00:00<00:00, 17242.77it/s]


Skipping 01__walking_down_indoor_hall_disgust.mp4 - already exists
Skipping 01__walk_down_hall_angry.mp4 - already exists
Skipping 01__outside_talking_still_laughing.mp4 - already exists
Skipping 01__talking_angry_couch.mp4 - already exists
Skipping 01__hugging_happy.mp4 - already exists
Skipping 01__walking_outside_cafe_disgusted.mp4 - already exists
Skipping 01__walking_down_street_outside_angry.mp4 - already exists
Skipping 01__talking_against_wall.mp4 - already exists
Skipping 01__podium_speech_happy.mp4 - already exists
Skipping 01__secret_conversation.mp4 - already exists
Skipping 01__meeting_serious.mp4 - already exists
Skipping 01__exit_phone_room.mp4 - already exists
Skipping 01__kitchen_still.mp4 - already exists
Skipping 01__kitchen_pan.mp4 - already exists
Skipping 01__walking_and_outside_surprised.mp4 - already exists
Skipping 01__outside_talking_pan_laughing.mp4 - already exists

Processing fake videos...
Found 16 videos in Dataset/FF++/fake


Processing videos: 100%|██████████| 16/16 [00:00<00:00, 18782.22it/s]

Skipping 01_27__outside_talking_still_laughing__ZYCZ30C0.mp4 - already exists
Skipping 01_21__walk_down_hall_angry__03X7CELV.mp4 - already exists
Skipping 01_11__secret_conversation__4OJNJLOO.mp4 - already exists
Skipping 01_27__walking_outside_cafe_disgusted__ZYCZ30C0.mp4 - already exists
Skipping 01_11__meeting_serious__9OM3VE0Y.mp4 - already exists
Skipping 01_11__walking_outside_cafe_disgusted__FAFWDR4W.mp4 - already exists
Skipping 01_20__outside_talking_still_laughing__FW94AIMJ.mp4 - already exists
Skipping 01_02__walk_down_hall_angry__YVGY8LOK.mp4 - already exists
Skipping 01_27__hugging_happy__ZYCZ30C0.mp4 - already exists
Skipping 01_03__podium_speech_happy__480LQD1C.mp4 - already exists
Skipping 01_20__outside_talking_pan_laughing__OTGHOG4Z.mp4 - already exists
Skipping 01_02__outside_talking_still_laughing__YVGY8LOK.mp4 - already exists
Skipping 01_03__hugging_happy__ISF9SP4G.mp4 - already exists
Skipping 01_03__talking_against_wall__JZUXXFRB.mp4 - already exists
Skipping 01

W0000 00:00:1739738048.693660  207051 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
